### Запуск QtLab

In [77]:
%run D:\qtlab_replacement\init.py
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
import pandas as pd
from save_pkl import *
%matplotlib qt5
from matplotlib import interactive
interactive(True)

In [78]:
import resonator_quality_factor_fit
import imp
imp.reload(resonator_quality_factor_fit)

<module 'resonator_quality_factor_fit' from 'D:\\qtlab_replacement\\scripts\\resonator_quality_factor_fit.py'>

In [79]:
pna = Agilent_N5242A('pna', address = 'PNA')

# Прицеливание

In [85]:
sample_name = 'Baumanka Vanya-3_0_0qubit'
min_freq = 6.0e9
max_freq = 7.0e9
data_root = get_location()

if not os.path.exists('{0}/{1}'.format(data_root[0], data_root[1])):
    os.mkdir('{0}/{1}'.format(data_root[0], data_root[1]))

if not os.path.exists('{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)):
    os.mkdir('{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name))

save_path = '{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)

In [86]:
pna.set_average(1)
pna.set_averages(1)
pna.set_xlim(min_freq, max_freq)
pna.set_nop(1001)
pna.set_bandwidth(100)
pna.set_power(-40)

True

In [87]:
fig, axes=plt.subplots(2,2, figsize=(8,8))
freqs=pna.get_freqpoints()
S21=pna.get_tracedata()
phase = np.unwrap(S21[1])
delay = np.diff(phase)/(freqs[1]-freqs[0])
delay_freqs = (np.asarray(freqs[1:])+np.asarray(freqs[:-1]))/2
axes[0,0].plot(freqs,10*np.log10(S21[0]**2))
axes[0,1].plot(freqs, phase)
axes[1,0].plot(np.real(S21[0]*np.exp(1j*S21[1])),np.imag(S21[0]*np.exp(1j*S21[1])))
axes[1,1].plot(delay_freqs, delay)
#axes[1,1].set_yscale('log')

In [88]:
sweep.sweep(pna, ([0], lambda x: 0, ''), filename="S21 overview "+sample_name)

A Jupyter Widget

Started at:  May 24 2018 16:58:38

Elapsed time: 0 h 0 m 9.37 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


{'S-parameter': (('Frequency',),
  (array([  6.00000000e+09,   6.00100000e+09,   6.00200000e+09, ...,
            6.99800000e+09,   6.99900000e+09,   7.00000000e+09]),),
  array([ 0.06680325+0.07593498j,  0.08721048+0.05076869j,
          0.09850368+0.02028761j, ...,  0.02898571-0.07092936j,
          0.00535210-0.07635351j, -0.01862666-0.07451353j]),
  {'log': 20},
  ('Hz',))}

In [35]:
threshold = 0.2

In [63]:
from scipy.signal import detrend
from scipy.signal import find_peaks_cwt
peaks_expected = 0.1
data = S21
data_points = len(data[0])
data_complex = 10**(data[0]/20)*np.exp(1j*detrend(np.unwrap(data[1])))
data_diff = np.abs(np.diff(data_complex))
data_diff[data_diff<threshold] = 0
plt.plot(freqs[1:], data_diff)
peaks = find_peaks_cwt(data_diff, np.linspace(1, 8, 4), noise_perc=100-(100.*peaks_expected)/data_points)
print ('found {0} peaks'.format(len(peaks)))
peak_freqs = freqs[1:][peaks]
for freq in peak_freqs:
    print (freq, ',')

found 510 peaks
6005000000.0 ,
6007000000.0 ,
6010000000.0 ,
6012000000.0 ,
6014000000.0 ,
6016000000.0 ,
6017000000.0 ,
6019000000.0 ,
6021000000.0 ,
6023000000.0 ,
6026000000.0 ,
6028000000.0 ,
6030000000.0 ,
6033000000.0 ,
6040000000.0 ,
6042000000.0 ,
6047000000.0 ,
6049000000.0 ,
6051000000.0 ,
6053000000.0 ,
6055000000.0 ,
6057000000.0 ,
6059000000.0 ,
6062000000.0 ,
6064000000.0 ,
6067000000.0 ,
6070000000.0 ,
6074000000.0 ,
6076000000.0 ,
6079000000.0 ,
6083000000.0 ,
6085000000.0 ,
6087000000.0 ,
6090000000.0 ,
6094000000.0 ,
6097000000.0 ,
6101000000.0 ,
6105000000.0 ,
6107000000.0 ,
6109000000.0 ,
6113000000.0 ,
6116000000.0 ,
6119000000.0 ,
6121000000.0 ,
6125000000.0 ,
6128000000.0 ,
6132000000.0 ,
6135000000.0 ,
6137000000.0 ,
6139000000.0 ,
6144000000.0 ,
6146000000.0 ,
6147000000.0 ,
6150000000.0 ,
6152000000.0 ,
6154000000.0 ,
6156000000.0 ,
6160000000.0 ,
6162000000.0 ,
6164000000.0 ,
6169000000.0 ,
6172000000.0 ,
6175000000.0 ,
6177000000.0 ,
6179000000.0 ,
618500000

C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_peak_finding.py:412: RuntimeWarning: divide by zero encountered in double_scalars
  snr = abs(cwt[line[0][0], line[1][0]] / noises[line[1][0]])
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_peak_finding.py:412: RuntimeWarning: invalid value encountered in double_scalars
  snr = abs(cwt[line[0][0], line[1][0]] / noises[line[1][0]])


# Мерить зонаторы

In [77]:
resonator_freqs = [ 
6.476e9,
6.611e9,
6.746e9,
7852500000.0]
powers = np.linspace(-30, -65, 36) # длинный скан
#powers = np.linspace(0, -40, 41) # короткий скан

In [42]:
resonator_freqs = [ 
6.6532e9
6.8037e9,    
]
#powers = np.linspace(-10, -45, 36) # длинный скан
powers = np.linspace(-20, -40, 41) # короткий скан

In [38]:
resonator_ids = [i for i in range(2)]
resonator_ids

[0, 1]

In [39]:
len(resonator_freqs)

2

In [43]:
resonator_freq_span = 10.0e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(100)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='reflection_port')
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

A Jupyter Widget

Started at:  May 23 2018 22:54:30

Elapsed time: 0 h 0 m 19.26 s


A Jupyter Widget

Started at:  May 23 2018 22:54:52

Elapsed time: 0 h 0 m 19.11 s


# load last measurements and refit

In [17]:
measurement[1]

':'

In [38]:
import glob
measurement_pattern = 'D:\\qtlab\\ReiData\\data\\2018-04-28\\*-CHeGe Vanya-3 res {}\\*.pkl'
for resonator_id in range(3):
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        #print (last_measurement)
        with open(last_measurement, 'rb') as f:
            measurement = pickle.load(f)[1]['S-parameter']
            powers = measurement[1][0]
            fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='reflection_port')
            fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
            plt.close()
            plt.close()

C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\utilities.py:9: RuntimeWarning: invalid value encountered in log10
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 800.
  warnings.warn(errors[info][0], RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 200.
  warnings.warn(errors[info][0], RuntimeWarning)


In [ ]:
resonator_freq_span = 1.0e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(10)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='reflection_port')
    fitdata['single_photon_limit'] =  port1.get_single_photon_limit()
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

# Загрузить фиты и построить графики

In [81]:
from matplotlib.backends.backend_pdf import PdfPages
fit_csvs = ['{0}/fit_total {1}.csv'.format(save_path, i) for i in range(len(resonator_freqs))]
rows = 3
columns = 2
figs, axes = plt.subplots(rows,columns, figsize=(20, 10))
pdf = PdfPages('{0}/{1}.pdf'.format(save_path, sample_name))
Q_i_values_lp = []
Q_i_values_hp = []
Q_e_values = []
Q_l_values_lp = []
Q_l_values_hp = []
f_r_values = []
single_photon_limits = []
resonator_type='reflection_port'
if resonator_type=='reflection_port':
    Qc_column = 'Qc'
    Qi_column = 'Qi'
    Qi_err_column = 'Qi_err'
    Ql_column = 'Ql'
else:
    Qc_column = 'Qc_dia_corr'
    Qi_column = 'Qi_dia_corr'
    Qi_err_column = 'Qi_dia_corr_err'
    Ql_column = 'Ql'
for resonator_id, fit_csv in enumerate(fit_csvs):
    try:
        fitdata = pd.read_csv(fit_csv)
        #print (fitdata.shape, powers.shape)
        #if fitdata.shape[0]<len(powers):
        #fitdata['power'] = powers[(len(powers)-fitdata.shape[0]):]
        #fitdata['power'] = powers
        print (fitdata, resonator_id)
        fitdata = fitdata[fitdata['fr']>4e9]
        fitdata = fitdata[fitdata['fr']<12e9]
        fitdata = fitdata[fitdata[Qc_column]>100]
        fitdata = fitdata[fitdata[Qc_column]<1000000]
        #if resonator_id != 11:
        #fitdata = fitdata[fitdata[Qc_column]<800000]
        #else:
        #   fitdata = fitdata[fitdata['Qc_dia_corr']<2000000]
        
        Q_ax = axes[(resonator_id)%rows, (resonator_id)//rows]
        Q_ax.set_yscale('log')
        Ql = Q_ax.errorbar(fitdata['power'], fitdata['Ql'], label='Q_l')#, yerr=fitdata['Ql_err'])
        Qc = Q_ax.errorbar(fitdata['power'], fitdata[Qc_column], label='Q_c')#, yerr=fitdata['absQc_err'])
        Qi = Q_ax.errorbar(fitdata['power'], np.abs(fitdata[Qi_column]), label='Q_i')#, yerr=fitdata['Qi_dia_corr_err'])
        f_ax = Q_ax.twinx()
        f = f_ax.errorbar(fitdata['power'], fitdata['fr']/1e6, color='orange', marker='.', label='$f$, MHz')#, yerr=fitdata['fr_err'])
        f_ax.set_ylabel('MHz')
        Q_ax.set_ylabel('$Q$')
        if not (fitdata['power'].size == 0):
            Q_i_values_lp.append(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])])
            Q_i_values_hp.append(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])])
            Q_e_values.append(np.median(fitdata[Qc_column]))
            Q_l_values_lp.append(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])])
            Q_l_values_hp.append(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])
            f_r_values.append(np.median(fitdata['fr']))
            single_photon_limits.append(fitdata['single_photon_limit'][np.argmax(fitdata['power'])])
            ann = Q_ax.annotate('$f_r$: {0:2.4} GHz, $Q_i$: {1:2.3g}/{3:2.3g}, $Q_l$:{5:2.3g}/{6:2.3g}, $Q_c$: {4:2.3g}'.format(np.median(fitdata['fr'])/1e9, \
                            np.round(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])]), np.round(fitdata[Qi_err_column][np.argmin(fitdata['power'])]), \
                            np.round(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])]), np.median(fitdata[Qc_column]), \
                            np.round(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])]), np.round(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])), \
                          (0.05, 0.1), xycoords='axes fraction', bbox={'alpha':0.5, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
        Q_ax.set_ylim([1e3, 1e6])
        if (resonator_id==0):
            leg = plt.legend([f, Qc, Qi, Ql], ['$f$', '$Q_c$', '$Q_i$', '$Q_l$'], fancybox=True, loc=6, framealpha=1)
            leg.get_frame().set_alpha(0.5)
            plt.title(sample_name)
    finally:
        pass
#axes[-1,-1].hist([Q_i_values_hp, Q_i_values_lp, Q_e_values], bins=10, label={'$Q_i$ high power', '$Q_i$ low power', '$Q_e$'})
#axes[-1,-1].set_xlabel('$Q_i$')
#axes[-1,-1].set_ylabel('resonators')
#axes[-1,-1].legend(loc='upper right')
plt.tight_layout()
pdf.savefig(figs)
pdf.close()
summary_table = pd.DataFrame({'fr':f_r_values, 'Ql_lp':Q_l_values_lp, 'Ql_hp':Q_l_values_hp, 'Qe': Q_e_values, 'Qi_lp':Q_i_values_lp, 'Qi_hp':Q_i_values_hp, 'single_photon_limits':single_photon_limits})
summary_table.to_csv('{0}/{1}.csv'.format(save_path, sample_name))

    Unnamed: 0            Qc        Qc_err            Qi        Qi_err  \
0            0  5.039733e+03  1.267889e+02  1.818691e+06  1.063848e+07   
1            1 -8.863986e+05  1.582121e+15 -1.710033e+08  1.054295e+22   
2            2  9.987763e+03  8.374421e+02  1.421253e+06  1.205278e+07   
3            3  3.931123e+07  4.803088e+08  8.255378e+08  6.109169e+12   
4            4  8.427305e+02  7.078806e+01  1.992692e+04  3.478339e+03   
5            5  3.718027e+03  8.057322e+02  3.618820e+03  1.303272e+03   
6            6  5.839104e+02  6.388468e+02  2.673404e+03  4.755493e+03   
7            7  7.431936e+02  1.011323e+03  2.683442e+03  1.267594e+03   
8            8  3.250927e+06  2.308842e+06  8.744297e+04  8.301975e+04   
9            9  7.079265e+02  2.737148e+02  6.297924e+03  4.517145e+03   
10          10  2.249748e+03  3.975073e+03  1.560928e+03  1.816243e+03   
11          11  1.467929e+03  1.882969e+03  2.149176e+03  2.945218e+03   
12          12  1.126759e+04  5.364405

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [78]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
(summary_table_new['Ql_lp']).round(decimals=0).astype(int)

0     88912
1     86737
2     67690
3     75804
4     76766
5     70556
6     48617
7     53069
8     36750
9     24874
10    31147
11    33566
12    27369
13    72075
14    25725
15    77900
16    37681
17    47956
18    62747
19    80936
20    54084
21    56429
22     8429
Name: Ql_lp, dtype: int32

In [79]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
summary_table_new

,Qe,Qi_hp,Qi_lp,Ql_hp,Ql_lp,fr,single_photon_limits,k
0,269731.318279,388738.656746,115631.358109,159456.126528,88912.285697,5.692377e+09,-154.453923,21103.879293
1,339969.028189,420879.707044,105819.168594,188737.010441,86736.758292,5.801470e+09,-154.730027,17064.701437
2,200201.756915,347295.123312,90651.902712,127199.604885,67689.860103,5.902237e+09,-153.469897,29481.446529
3,457301.001096,302781.321422,92481.453827,160244.927163,75803.574278,6.002529e+09,-153.035384,13125.990296
4,233119.447301,318540.813969,99559.922172,131852.909449,76765.951591,6.114608e+09,-152.979287,26229.508328
5,444415.955726,283448.830382,82511.072962,163763.547560,70556.242405,6.210245e+09,-152.361917,13973.946093
6,248919.297185,147412.743270,60468.888851,92566.983543,48617.429598,6.308563e+09,-149.198303,25343.807188
7,590040.532930,245132.211818,58598.210671,175700.739158,53068.793810,6.424510e+09,-150.638738,10888.251180
8,78909.143205,272204.327372,69003.484245,61108.177275,36749.634389,6.515155e+09,-150.304592,82565.273061
9,38139.877457,265438.836812,68127.195368,33351.374869,24873.539070,6.637037e+09,-148.034765,174018.309409


# bullshit

In [23]:
freqs=measurement.get_data()[1]
powers=measurement.get_data()[0]
max_power_S21 = 10**(measurement.get_data()[2][np.argmax(powers), :]/20)*np.exp(1j*measurement.get_data()[3][np.argmax(powers), :])

fitter = circuit.reflection_port(f_data=freqs, z_data_raw=max_power_S21)
print(fitter.do_calibration(freqs, max_power_S21))
fitter.autofit()
fitter.plotall()
print(fitter.fitresults)
print(fitter.get_single_photon_limit())

(8.7617753961513634e-08, 0.00021846153804807672, 0.78230385058357799, 9473514161.3102646, 987237.85108576983, 0.0, 9480474204.3878784)
{'theta0': -0.87036269078204709, 'Qi_err': 4279766.7041571699, 'Ql_err': 109.19720855604572, 'Qc_err': 91.415426058562019, 'fr': 9482648929.5436897, 'Qc': 763.26702083792111, 'Qi': 198307.89384138637, 'fr_err': 516643.97835309943, 'Ql': 760.34054699522324, 'chi_square': 1.0765021049802501}
-129.081045603


In [6]:
measurement = dm.load_measurement('C:/qtlab/ReiData/data/Feb 27 2017/16-17-54/Res Nb CEXP 17.01.2016 switch 4-2 SH A anti-korenkovian warm')